In [1]:
# Import packages

import numpy as np
import torch
import pandas as pd
import plotly.express as px

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
from datasets import load_dataset

from tqdm.notebook import trange, tqdm
from ipywidgets import IntProgress

2023-02-23 16:20:14.338656: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-23 16:20:14.521640: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-23 16:20:15.310805: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-23 16:20:15.310898: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:
# Import pretrained BERT finetuned on IMDB Database

tokenizer = AutoTokenizer.from_pretrained("fabriceyhc/bert-base-uncased-imdb")

model = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-imdb")

In [3]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# Loading IMDB dataset (IN) and SST2 (OUT)

dataset = load_dataset("imdb")

out_dataset = load_dataset("sst2")

## Generate train distribution

In [ ]:
# Hook functions to get latent representation of data

def features_hook_0(model, inp, output):
    global feat_0
    feat_0 = output

def features_hook_1(model, inp, output):
    global feat_1
    feat_1 = output

def features_hook_2(model, inp, output):
    global feat_2
    feat_2 = output

def features_hook_3(model, inp, output):
    global feat_3
    feat_3 = output

def features_hook_4(model, inp, output):
    global feat_4
    feat_4 = output

def features_hook_5(model, inp, output):
    global feat_5
    feat_5 = output

def features_hook_6(model, inp, output):
    global feat_6
    feat_6 = output

def features_hook_7(model, inp, output):
    global feat_7
    feat_7 = output

def features_hook_8(model, inp, output):
    global feat_8
    feat_8 = output

def features_hook_9(model, inp, output):
    global feat_9
    feat_9 = output

def features_hook_10(model, inp, output):
    global feat_10
    feat_10 = output

def features_hook_11(model, inp, output):
    global feat_11
    feat_11 = output

features_hooks = [features_hook_0, features_hook_1, features_hook_2, 
                    features_hook_3, features_hook_4, features_hook_5, 
                    features_hook_6, features_hook_7, features_hook_8, 
                    features_hook_9, features_hook_10, features_hook_11]



feat_hook = [model.base_model.encoder.layer[i].register_forward_hook(features_hooks[i]) for i in range(12)]

In [ ]:
def get_lattent_representation(input_data, model):
    """aggregating latent representation to create a unique representation vector
    """
    pipe = TextClassificationPipeline(
                                model=model, tokenizer=tokenizer
                                )
    pipe(input_data)
    feats = [feat_0[0], feat_1[0], feat_2[0], 
                feat_3[0], feat_4[0], feat_5[0], 
                feat_6[0], feat_7[0], feat_8[0], 
                feat_9[0], feat_10[0], feat_11[0]]
    aggregated_features = torch.mean(torch.stack([i[0, 0, :] for i in feats]), dim= 0)

    return aggregated_features

###### IF THE DISTRIBUTION IS NOT COMPLETED ######

# distrib =[] 
# fail = []
# for i in tqdm(range(len(dataset['train']))):
#     if i%100 == 0:
#         print(i)
#     try:
#         distrib.append(get_lattent_representation(dataset['train'][i]['text'], model))
#     except:
#         fail.append(i)

# def process_distrib(distrib):
#     return np.vstack([distrib[i].numpy().flatten() for i in range(len(distrib))])

# distrib = process_distrib(distrib)
# pd.DataFrame(distrib).to_csv('distrib_.csv', index = False)




###### IF THE DISTRIBUTION IS ALREADY COMPLETED ######


distrib = np.array(pd.read_csv('distrib_first.csv'))
# distrib = distrib[:, 1:]
assert distrib.shape[1] == 768 # checking the shape : vectors must have a 768 size

In [ ]:
def get_logits(input_data, model):
    pipe = TextClassificationPipeline(
                                model=model, tokenizer=tokenizer
                                )
    res = pipe(input_data)
    return res[0]['score']


In [ ]:
def get_lattent_last_layer(input_data, model):
    pipe = TextClassificationPipeline(
                            model=model, tokenizer=tokenizer
                                )
    pipe(input_data)
    feat = feat_11[0]

    return feat[0][0]

###### IF THE DISTRIBUTION IS NOT COMPLETED ######

distrib_last_layer = [] 
fail = []
for i in tqdm(range(len(dataset['train']))):
    if i%100 == 0:
        print(i)
    try:
        distrib_last_layer.append(get_lattent_last_layer(dataset['train'][i]['text'], model))
    except:
        print(i)
        fail.append(i)

def process_distrib(distrib):
    return np.vstack([distrib[i].numpy().flatten() for i in range(len(distrib))])

distrib_last_layer = process_distrib(distrib_last_layer)
pd.DataFrame(distrib_last_layer).to_csv('distrib_last_layer.csv', index = False)




###### IF THE DISTRIBUTION IS ALREADY COMPLETED ######


distrib_last_layer = np.array(pd.read_csv('distrib_last_layer.csv'))
# distrib = distrib[:, 1:]
assert distrib.shape[1] == 768 # checking the shape : vectors must have a 768 size

## Distance functions

In [ ]:
# computing variables for Mahalanobis score

cov_matrix = np.cov(distrib.T)
precision_matrix = np.linalg.inv(cov_matrix)
esperance_vector = np.mean(distrib, axis = 0)

def D_M(x, esperance = esperance, precision = precision_mat):
    """Mahalanobis distance
    """
    v = x - esperance
    u = 1 + (v.T @ precision @ v)
    return -1 / u

In [ ]:
def MSP(x):
    """Max softmax proba
    x is the max logits value
    """
    return 1 - x

In [ ]:
df = pd.read_csv('lilian_scripts/data/train_latent.csv')

In [ ]:
df = df.T
predicted_0 = df[df[9216] == 0].T
predicted_1 = df[df[9216] == 1].T

In [ ]:
predicted_0_vectors = 
predicted_1_vectors = 

In [ ]:
class_0_variables = (np.mean(distrib, axis = 0) ,np.linalg.inv(np.cov(distrib.T)))

In [ ]:
# Last layer mahalanobis score

cov_matrix = np.cov(distrib.T)
precision_matrix = np.linalg.inv(cov_matrix)
esperance_vector = np.mean(distrib, axis = 0)

def D_M(x, esperance = esperance, precision = precision_mat):
    """Mahalanobis distance
    """
    v = x - esperance
    u = 1 + (v.T @ precision @ v)
    return -1 / u

## Computing In and Out distances 

In [ ]:
N = min([len(out_dataset['test']), len(dataset['test'])])

In [ ]:
N = 10

In [ ]:
distances = {
            'maha' : {'in' : [], 'out' : []},
            'MSP' : {'in' : [], 'out' : []},
            }

for i in tqdm(range(N)):
    input_data = out_dataset['test'][i]['sentence']

    x = get_lattent_representation(input_data, model)
    distances['maha']['out'].append(D_M(x.numpy()))

    logit = get_logits(input_data, model)
    distances['MSP']['out'].append(MSP(logit))

for i in tqdm(range(N)):
    try:
        input_data = dataset['test'][i]['text']

        x = get_lattent_representation(input_data, model)
        distances['maha']['in'].append(D_M(x.numpy()))

        logit = get_logits(input_data, model)
        distances['MSP']['in'].append(MSP(logit))

    except:
        pass

## Results

In [ ]:
px.histogram(
            distances['maha']['out'] + distances['maha']['in'], 
            color = ['out'] * len(distances['maha']['out']) + ['in'] * len(distances['maha']['in']), 
            template = 'none'
            )

In [ ]:
def error(in_distances, out_distances, threshold):
    v = np.sum([i >= threshold for i in in_distances]) + np.sum([i < threshold for i in out_distances])
    return v / (len(out_distances) + len(in_distances))

def compute_best_threshold(in_distances, out_distances, n_candidates = 1000):

    candidates = np.linspace(min(in_distances), max(out_distances), n_candidates)

    errors = [error(in_distances, out_distances, i) for i in candidates]
    best_index = np.argmin(errors)
    return (candidates[best_index], errors[best_index])


In [ ]:
compute_best_threshold(distances['maha']['in'], distances['maha']['out'])

In [ ]:
compute_best_threshold(distances['MSP']['in'], distances['MSP']['out'])

## Some tries

In [ ]:
def generate_sphere_point(ndim):
    vec = np.random.randn(ndim)
    vec /= np.linalg.norm(vec, axis=0)
    return vec

def compute_minimum_value(x, distrib, u_k):
    positive_rate = np.mean(np.array([np.dot(u_k, distrib[i] - x) for i in range(len(distrib))]) > 0)
    return min(positive_rate, 1 - positive_rate)

def D(x, distrib, n_proj = 10):
    u = [generate_sphere_point(x.shape[0]) for _ in range(n_proj)]
    vector_of_minimums = [compute_minimum_value(x, distrib, u_k) for u_k in u]
    return np.mean(vector_of_minimums)

In [ ]:
def D_(x, distrib, n_proj = 100):
    u = [generate_sphere_point(x.shape[0]) for _ in range(n_proj)]
    vector_of_minimums = [compute_minimum_value(x, distrib, u_k) for u_k in u]
    return (vector_of_minimums)

x = np.random.random(distrib[0].shape)
a = D_(x, distrib, 1000)

px.line([np.mean(a[:i]) for i in range(1, len(a))], template = 'none')